<div align="center">
    <h1><strong>Software de diseño automático</strong></h1>
</div>

---

<h2 style="text-align:center"><strong>Resumen</strong></h2>
<div style="text-align:justify">
    <br>
    Se desarrolló una metodología de diseño automático del sistema de elución y filtrado que simula el comportamiento fluidodinámico durante la etapa de filtrado, permitiendo predecir el grado de concentración de partículas a lo largo del sistema a través de un modelo numérico basado en CFD-DEM. Esta metodología ha sido elaborada con herramientas de código abierto. Utilizando <i>Python</i> como lenguaje base, <i>Jupyter</i> como entorno de desarrollo, <i>ParaView</i> como plataforma de análisis de resultados y librerías de <i>C++</i> (como Yade, LIGGGHTS y OpenFoam) para el desarrollo de las simulaciones numéricas.
</div>

<br>

___Palabras clave:___ CFD-DEM, Python, Jupyter, ParaView, OpenFoam.

---

# __5. Diseño del sistema__

El diseño del sedimentador de placas paralelas se resume en lo siguiente:

1. Definición de propiedades del _fluido_ y _material particulado_.
2. Cálculo de las propiedades termodinámicas del fluido.
3. Definición de la geometría del panel de lamelas.
4. Cálculo de propiedades de flujo.
5. Resultados.

## 5.1. Generalidades

Define las propiedades del _fluido_ y _material particulado_:

In [ ]:
from App.Generalidades.Data import *
data = Datos()
data

Con base en los datos suministrados, las propiedades termodinámicas del solvente se pueden apreciar a continuación.

In [ ]:
from App.Generalidades.Read import Read
from App.Generalidades.Propiedades import *
general = Read(data)
prop = SolProp(general)()
prop = prop['mezcla']

## 5.2. Naturaleza del flujo

El éxito de un sistema de sedimentación recae en la naturaleza del flujo durante el proceso de separación de sustancias. Siendo necesario que éste sea de carácter laminar para evitar reflujos. 

Asumiendo régimen laminar en la interacción _fluido - partícula_, la velocidad de sedimentación se calcula a través de la siguiente relación matemática.

$$
\begin{equation}
    V_{laminar} = \frac{g}{18} \left(S_s -1 \right) \, \frac{d_{particula} ^2}{\mu}
\end{equation}
$$

Con base en este valor de velocidad, el número de Reynolds se calcula de la siguiente forma:

$$
\begin{equation}
    Re = \frac{V_{laminar} \, d_{particula}}{\mu}
\end{equation}
$$

In [ ]:
from App.Sedimentation.Inicial import *
Q = caudal(general['Fluido']['Volumen [L]'], general['Fluido']['Tiempo objetivo [h]'])
res_inicial = ReyLaminar(general, prop, Q)()

## 5.3. Dimensionamiento

A continuación, se define la geometría del panel de lamelas para el diseño del sistema de sedimentación. 

In [ ]:
from App.Generalidades.Lamelas import *
geo = Geometry()
geo 

In [ ]:
from App.Generalidades.Read import *
datos_geo = Read(geo, datos={'lamela':{}})
datos_geo = datos_geo['lamela']
lamela = Flujo_int(datos_geo, general)

La velocidad crítica de sedimentación de las partículas se calcula a partir de la Ecuación \ref{yao}.

$$
\begin{equation}
    V_{sc} = \frac{S_c \, V_0}{\sin \theta + L_{rel} \cos \theta}
    \label{yao}
    \tag{1}
\end{equation}
$$

De la Ecuación \ref{yao}, $S_c$ es una cosntante de sedimentación, cuyo valor es 1 para sistemas de placas paralelas, $V_0$ es la magnitud del flujo, $L_{rel}$ es la relación entre la longitud de una lamela y el ancho de la misma; y $\theta$ es el ángulo de inclinación de las placas.

La relación entre el ancho $e$ del conducto y la longitud $L$ tiene una importancia especial en la eficiencia de un sedimentador. Si esta relación $L_{rel}$ es muy pequeña, cada sedimentdor actúa como un sedimentador horizontal de corriente de baja velocidad.



Para que un sedimentador pueda trabajar con alta vleocidad, es necesario que exista flujo laminar en las celdas, esto es que el número de Reynoles sea inferior a 250. Cualquier turbulencia puede generar arrastre de partículas, bajando notoriamente la eficiencia; razón por la que se emplea como criterio principal de diseño.

$$
\begin{equation}
    Re = \frac{4 \, V_0 \, R_h}{\mu}
    \label{Re_def}
    \tag{2}
\end{equation}
$$

En dónde $R_h$ es el radio hidráulico, cuyo valor está definido en la Ecuación \ref{rh}.

$$
\begin{equation}
    R_h = \frac{b \, e}{2 \left(b + e \right)}
    \label{rh}
    \tag{3}
\end{equation}
$$

In [ ]:
from App.Sedimentation.ParOper import *
res_oper = Parametros(general, prop, lamela, datos_geo)()

## 5.4. Velocidad de sedimentación

Conociendo las propiedades del solvente y del material particulado, se puede calcular la _velocidad de sedimentación_ de las partículas a parti de la Ecuación \ref{Vsed}.

$$
\begin{equation}
    V_{sed} = 0.22 \left(g \frac{\rho _{sol} - \rho _{liq}}{\rho _{liq}} \right) ^{2/3} \frac{d_{part}}{\sqrt[3]{\frac{\mu}{\rho _{liq}}}}
    \label{Vsed}
    \tag{4}
\end{equation}
$$

La eficiencia del sistema se calcula empleando la Ecuación \ref{Ef}.

$$
\begin{equation}
    Ef = 1-\frac{V_{sc}}{V_{sed}}
    \label{Ef}
    \tag{5}
\end{equation}
$$

In [ ]:
from App.Sedimentation.Resultados import *
res_sed = Resul(prop, res_oper['V'], general['Solid'] )()

## 5.5. Método de Hazen $\rightarrow$ tiempo de sedimentación

El concepto de carga hidráulica superficial (parámetro de Hazen) compara la
velocidad de sedimentación de la partícula para conocer si alcanza a sedimentarse
durante el trayecto por el sistema. La altura de sedimentación requerida se calcula
mediante la Ecuación \ref{Hsed}.

$$
\begin{equation}
    H_{sedimentacion} = \frac{e}{\cos \theta}
    \label{Hsed}
    \tag{6}
\end{equation}
$$

El tiempo requerido para que la sedimentación se desarrolle se calcula de la siguiente forma:

$$
\begin{equation}
    t_{sed} = \frac{H_{sedimentacion}}{V_{sed}}
    \tag{7}
\end{equation}
$$

In [ ]:
Hazen = Tiempo(datos_geo['Ancho lamela [cm]'], datos_geo['Inclinación [°]'], res_sed['V_sed'])()

## 5.6. Resultados

El resumen de resultados obtenidos se pueden apreciar en la siguiente tabla:

In [ ]:
TablaResumen(lamela, res_oper, res_sed, Hazen)

# __6. Modelo CFD-DEM__

El procedimiento para el planteamiento del modelo CFD-DEM consiste en lo siguiente:
1. Definición de la geometría del sistema.
2. Mallado de la geometría.
3. Definición de condiciones de frontera
4. Desarrollo de la simulación.
5. Postprocesamiento y presentación de resultados.
6. Análisis de resultados.



## 6.1. Geometría y mallado

Para definir la geometría del sistema, es necesario garantizar régimen laminar en todo el _volumen de control_. Para ello, la velocidad del flujo en la tubería de ingreso debe tener un número de Reynolds igual o inferior a 250. El diámetro de tubería que garantiza un régimen laminar está definido por la Ecuación \ref{DT}

$$
\begin{equation}
    D_I = \frac{4 \, Q_T}{Re \, \pi \mu}
    \tag{8}
    \label{DT}
\end{equation}
$$

In [ ]:
from IPython.display import display, Markdown
from math import pi
datos_geo['DI [m]'] = round((2*lamela['Q_T'])/(125*pi*prop['visc']),2)
display(Markdown("El diámetro de la tubería de ingreso tiene un valor de $D_I = DT [cm]$".replace("DT", str(datos_geo['DI [m]']*100))))


El mallado consiste en al discretizaciòn del dominio geométrico del problema.

<div align="center">
    <img src="Images/Elemento.png" style="width: 300px;" />
</div>
<div align="center">
    <i>Figura 2.</i> Discretización del dominio.
</div>

El mallado de la geometría se especifica a continuación.

In [ ]:
from App.CFD_DEM.Geom import *
from App.Generalidades.Read import *
from ipywidgets import Button, Layout, HBox
size = Malla(round((2*lamela['Q_T'])/(125*pi*prop['visc']),2))
a = HBox([size], layout=Layout(height='200px', overflow_y='auto'))
display(a)

In [ ]:
import warnings
datos_geo = ReadMesh(size)
datos_geo['DI [m]'] = round((2*lamela['Q_T'])/(125*pi*prop['visc']),2)
warnings.filterwarnings('ignore')
DEM = False      #Cambiar modelo de malla en el directorio CFD-DEM
Dibujar(datos_geo, lamela, size.children[0].children[0].value, size.children[0].children[1].value, general['Solid']['Tamaño de partícula medio [um]'], (size.children[2].children[0].value, size.children[2].children[1].value), DEM)

La malla desarrollada presenta las siguientes características:

In [ ]:
import pyvista as pv
res_malla = pv.read("out.vtk")
res_malla

Resultados de la evaluación de malla a través del comando `checkMesh`, de OpenFoam:

In [ ]:
from App.CFD_DEM.Malla import *
ToFoam()

## 6.2. Condiciones de frontera

Como se aprecia en la Figura 2, el volumen de control presenta una _entrada_ y una _salida_. También cuenta con una zona de lodos, en donde se deposita parte del material particulado que se busca separar. Las condiciones de frontera, que son tanto de tipo _Dirichlet_ y _Neumann_, se pueden apreciar a continuación.

<div align="center">
    <img src="Images/CF.PNG" style="width: 250px;" />
</div>
<div align="center">
    <i>Figura 2.</i> Discretización del dominio bidimensional.
</div>



In [ ]:
from App.CFD_DEM.CF import *
Condiciones(lamela['Q_T'], size.children[2].children[0].value)

Cabe resaltar que de las condiciones de frontera, se emplea la condición de _no deslizamiento_ en los muros del sistema. Como se muestra a continuación, en ``ParaView``.

In [ ]:
import os 
os.system('paraview "./OpenFOAM/system/controlDict"')

## 6.3. Desarrollo de la simulación

La simulación numérica a realizar comprende dos métodos numéricos: el método de _volúmenes finitos_ (FVM, por sus siglas en inglés), con el cual se predice el comportamiento fluidodinámico dentro del volumen de control, y el método de _elementos discretos_ (DEM, por sus siglas en inglés), con el que se predice el comportamiento dinámico de las partículas sólidas y su interacción con el solvente.



### 6.3.1. CFD
La _Dinámica de Fluidos Computacional_ (CFD) es una herramienta computacional ampliamente usada en ingeniería para el desarrollo de simulaciones numéricas que involucren fluidos. Emplea como método base el método de volúmenes finitos (FVM). Este método numérico transforma las ecuaciones diferenciales parciales, que representan las leyes conservativas, en ecuaciones algebraicas discretas sobre volúmenes finitos.

Se trata de un problema _bidimensional_ (2D); de modo que la discretización del dominio se plantea de de la siguiente forma:

<div align="center">
    <img src="Images/malla.png" style="width: 250px;" />
</div>
<div align="center">
    <i>Figura 3.</i> Discretización del dominio bidimensional.
</div>

El diagrama de flujo de la solución del problema se puede apreciar a continuación.

<div align="center">
    <img src="Images/flujo.png" style="width: 350px;" />
</div>
<div align="center">
    <i>Figura 4.</i> Diagrama de flujo del solucionador pimpleFoam.
</div>

En donde, para un modelo de flujo laminar, la Ecuación _general_ del problema bidimensional transitorio que se busca resolver se puede apreciar en la Ecuación \ref{2D}.

$$
\begin{equation}
    \underbrace{\rho \frac{\partial \phi}{\partial t}}_{\text{transitorio}} + \underbrace{\rho u \frac{\partial \phi}{\partial x} + \rho v \frac{\partial \phi}{\partial y}}_{\text{convectivo}} = \underbrace{ \frac{\partial}{\partial x} \left( \Gamma \frac{\partial \phi}{\partial x} \right) + \frac{\partial}{\partial y} \left( \Gamma \frac{\partial \phi}{\partial y} \right)}_{\text{difusivo}} + \underbrace{S_{\phi}}_{\text{fuente}}
    \tag{9}
    \label{2D}
\end{equation}
$$

El _comportamiento_ fluidodinámico del problema, resuleto mediante ``OpenFOAM``, se puede observar a continuación a través de la ejecución del siguiente algoritmo. 

In [ ]:
from App.CFD_DEM.CFD import *
FOAM()

### __6.3.2 CFD-DEM__

Una vez observado el fenómeno fluido sin material particulado, se procede a emplear un modelo CFD-DEM capaz de predecir el comportamiento dinámico del material particulado durante el proceso de separación de sustancias.


<div align="center">
    <img src="Images/CFDEM.png" style="width: 900px;" />
</div>
<div align="center">
    <i>Figura 5.</i> Metodología del modelo CFD-DEM.
</div>

In [ ]:
#ToFoam(dir="CFD_DEM", imprime=False)
#Condiciones(lamela['Q_T'], size.children[2].children[0].value, dir="CFD_DEM/", imprime = False)

#¿Nueva simulación?
#clean("CFD_DEM") 

Se empleó un enfoque _Eulerian - Lagrange_ (E-L) para resolver el problema, debido a que adopta la teoría continua para la fase líquida y resuelve el modelo dinámico de cada __partícula individual__. Gracias a ello, información referente a la posición, velocidad, fuerza hidrodinámica y difusividad, entre otros, se pueden obtener para cualquier insante de tiempo. Teniendo así el potencial de sobrellevar las dificultades y limitaciones inherentes en modelos _teóricos_ y _empíricos_.

<div align="center">
    <img src="Images/particulas.png" style="width: 700px;" />
</div>
<div align="center">
    <i>Figura 6.</i> Dinámica de las partículas.
</div>

$$
\begin{equation}
	m_i \frac{d v _i}{dt} = F_{c,i} + F_{f,i} + F_{g,i}
	\label{DEMdyn}
    \tag{10}
\end{equation}
$$


$$
\begin{equation}
	I_i \frac{d \omega _i}{dt} = T_{c,i} + T_{r,i}
	\label{DEMrot}
    \tag{11}
\end{equation}
$$

Por tratarse de partículas con tamaños del orden de los micrómetros, se requiere de un _alto costo computacional_ para llevar a cabo una simulación.  

In [ ]:
from App.Postprocessing.part import *
particulas()